### Tools are the python functons 

In [28]:
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, ToolMessage
from langgraph.graph import START, END, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated, Sequence
from langchain_core.tools import tool


### Lets create 3 tools.

In [29]:
@tool
def add(a:int, b:int):
    ''' This function will add given 2 numbers'''

    return a + b

@tool
def sub(a:int, b:int):
    ''' This function will sub given 2 numbers'''

    return a - b


@tool
def mul(a:int, b:int):
    ''' This function will multiply given 2 numbers'''

    return a * b

### Bind above tools to LLM


In [30]:
from gen_ai_hub.proxy.langchain.init_models import init_llm

model = init_llm(model_name = "gpt-4o")
tools = [add, sub, mul]
model.bind_tools(tools=tools)

RunnableBinding(bound=ChatOpenAI(client=<gen_ai_hub.proxy.native.openai.clients.ChatCompletions object at 0x000002A79965EA90>, async_client=<gen_ai_hub.proxy.native.openai.clients.AsyncChatCompletions object at 0x000002A79962A510>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), n=1, top_p=1.0, max_tokens=256, proxy_client=GenAIHubProxyClient(base_url=None, auth_url=None, client_id=None, client_secret=None, resource_group=None, ai_core_client=<ai_core_sdk.ai_core_v2_client.AICoreV2Client object at 0x000002A7F8C8A190>), deployment_id='dc5b752a11e5fa73', config_name='gpt-4o', config_id='1066614d-388f-4f86-8dcb-e8e37ed1fbe1', proxy_model_name='gpt-4o'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': 'This function will add given 2 numbers', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 

In [43]:
model.invoke("Add 40 + 12 and then multiply the result by 6. Also tell me a joke please.").content

"First, let's add 40 and 12:\n\n40 + 12 = 52\n\nNext, let's multiply the result by 6:\n\n52 * 6 = 312\n\nSo, the final result is 312.\n\nNow, here's a joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"

In [32]:
class AgentState(TypedDict):
    message: Annotated[Sequence[BaseMessage], add_messages]

### Build a node

In [33]:
def model_call(state:AgentState) -> AgentState:
    system_input = SystemMessage(content="You are a helpfull AI, answer the question to your highest ability!")
    response = model.invoke([system_input] + state["message"])
    return {"message" : [response]}

### Build a looping router

In [34]:
def should_continue(state:AgentState):
    messages = state['message']
    last_message = messages[-1]
    if not last_message.tool_calls:
        return "end"
    else:return "tools"
    

### Build a toolnode
- Tool node is a node which contains all type of tools u created


In [36]:
toolnode = ToolNode(tools=tools)

graph_builder =StateGraph(AgentState)
graph_builder.add_node("agent", model_call)
graph_builder.add_node("tools", toolnode)

graph_builder.add_edge(START, "agent")

graph_builder.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        "end":END
    }

)

graph_builder.add_edge("tools", "agent")
app = graph_builder.compile()


In [42]:
def print_stream(stream):
    for s in stream:
        message = s["message"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


# inputs = {
#     "messages": [HumanMessage(content="Add 40 + 12 and then multiply the result by 6. Also tell me a joke please.")]
# }

inputs = {"message": [("user", "Add 40 + 12 and then multiply the result by 6. Also tell me a joke please.")]}
print_stream(app.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Add 40 + 12 and then multiply the result by 6. Also tell me a joke please.
================================== Ai Message ==================================

First, let's add 40 and 12:

40 + 12 = 52

Next, let's multiply the result by 6:

52 * 6 = 312

So, the final result is 312.

Now, here's a joke for you:

Why don't scientists trust atoms?

Because they make up everything!
